In [ ]:
# ===================================================================
# PART 0: INSTALL NECESSARY LIBRARIES
# ===================================================================
# This will correctly install the libraries for GARCH models and the DM test
!pip install arch dieboldmariano --quiet

import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import warnings
import io
import random

# <<< FIX: Correctly import the DM test from the library we just installed >>>
from dieboldmariano import dm_test

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Scikit-learn imports
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score

# TensorFlow imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Layer
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K

# Arch imports
from arch import arch_model

# ===================================================================
# PART 1: SETUP AND DATA LOADING
# ===================================================================
# --- 1.1 Set Random Seed for Reproducibility ---
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
print("Global random seeds set.")
print("\n" + "="*50)
print("IMPORTANT: FOR FASTER TRAINING, GO TO 'RUNTIME' > 'CHANGE RUNTIME TYPE'")
print("AND SELECT 'T4 GPU' AS THE HARDWARE ACCELERATOR.")
print("="*50 + "\n")

# --- 1.2 Load Data ---
print("Please upload your 'final_predictors_2003_onwards.csv' file.")
uploaded = files.upload()
filename = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[filename]), index_col='Date', parse_dates=True)
print("\n--- Data Loaded Successfully ---")

# ===================================================================
# PART 2: FEATURE ENGINEERING & TARGET VARIABLE
# ===================================================================
df['USDINR_log_return'] = np.log(df['USDINR'] / df['USDINR'].shift(1))
df['SP500_log_return'] = np.log(df['SP500'] / df['SP500'].shift(1))
df['OIL_WTI_log_return'] = np.log(df['OIL_WTI'] / df['OIL_WTI'].shift(1))
df['GOLD_log_return'] = np.log(df['GOLD'] / df['GOLD'].shift(1))
df['REALIZED_VOLATILITY'] = df['USDINR_log_return'].rolling(window=21).std() * np.sqrt(252)
df['RV_d'] = df['REALIZED_VOLATILITY'].shift(1)
df['RV_w'] = df['REALIZED_VOLATILITY'].shift(1).rolling(window=5).mean()
df['RV_m'] = df['REALIZED_VOLATILITY'].shift(1).rolling(window=22).mean()
df.dropna(inplace=True)

# --- Define Feature Sets ---
y = df['REALIZED_VOLATILITY']
X_full = df.drop(columns=['REALIZED_VOLATILITY', 'USDINR', 'SP500', 'OIL_WTI', 'GOLD', 'USDINR_log_return'])
X_simple = df[['RV_d', 'RV_w', 'RV_m']]

# ===================================================================
# PART 3: MODEL DEFINITIONS
# ===================================================================
# --- 3.1 LSTM Sequence Creation ---
def create_sequences(X_data, y_data, window_size):
    X_seq, y_seq = [], []
    for i in range(len(X_data) - window_size):
        X_seq.append(X_data[i:(i + window_size)])
        y_seq.append(y_data[i + window_size])
    return np.array(X_seq), np.array(y_seq)

# --- 3.2 LSTM Model Builder ---
class Attention(Layer):
    def __init__(self, **kwargs): super(Attention, self).__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1), initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1), initializer='zeros', trainable=True)
        super(Attention, self).build(input_shape)
    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b); e = K.squeeze(e, axis=-1)
        alpha = K.softmax(e); alpha = K.expand_dims(alpha, axis=-1)
        context = x * alpha
        return K.sum(context, axis=1)

def build_lstm_model(window_size, n_features):
    tf.random.set_seed(seed_value)
    input_layer = Input(shape=(window_size, n_features))
    lstm1 = LSTM(64, return_sequences=True)(input_layer)
    dropout1 = Dropout(0.2)(lstm1)
    lstm2 = LSTM(32, return_sequences=True)(dropout1)
    attention = Attention()(lstm2)
    dense1 = Dense(20, activation='relu')(attention)
    output_layer = Dense(1)(dense1)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# ===================================================================
# PART 4: WALK-FORWARD VALIDATION (MODIFIED FOR SPEED)
# ===================================================================
print("\n--- Starting Walk-Forward Validation (PROTOTYPING MODE) ---")

# --- 4.1 Define Split Parameters ---
split_index = int(len(df) * 0.8)
train_df = df.iloc[:split_index]
test_df = df.iloc[split_index:]

n_folds = 14
WINDOW_SIZE = 60
EPOCHS_PROTOTYPE = 50

n_test = len(test_df)
FORECAST_CHUNK_SIZE = int(np.ceil(n_test / n_folds))
print(f"Test set size: {n_test} days. Forecasting in {n_folds} folds of ~{FORECAST_CHUNK_SIZE} days each.")

# --- 4.2 Create Lists to Store Results ---
all_har_preds, all_garch_preds, all_gjr_preds, all_lstm_full_preds, all_lstm_simple_preds = [], [], [], [], []
all_actuals = []

# --- 4.3 Start the Walk-Forward Loop ---
for i in range(n_folds):
    print(f"\n--- Fold {i+1}/{n_folds} ---")
    start_idx = i * FORECAST_CHUNK_SIZE
    end_idx = min((i + 1) * FORECAST_CHUNK_SIZE, n_test)

    current_test_chunk = test_df.iloc[start_idx:end_idx]
    if current_test_chunk.empty: continue
    current_train_df = df.iloc[:split_index + start_idx]

    y_test_fold = y.loc[current_test_chunk.index]
    all_actuals.append(y_test_fold)

    # --- 4.3.1 HAR-RV Model ---
    X_har_train = X_simple.loc[current_train_df.index.intersection(X_simple.index)]
    y_har_train = y.loc[X_har_train.index]
    X_har_test = X_simple.loc[current_test_chunk.index.intersection(X_simple.index)]

    if not X_har_test.empty:
        har_model = LinearRegression()
        har_model.fit(X_har_train, y_har_train)
        har_preds = har_model.predict(X_har_test)
        all_har_preds.append(pd.Series(har_preds, index=X_har_test.index, name="HAR_Forecast"))

    # --- 4.3.2 GARCH Models ---
    returns_loop = current_train_df['USDINR_log_return'].dropna() * 100
    horizon = len(current_test_chunk)
    garch_model = arch_model(returns_loop, vol='Garch', p=1, q=1); garch_res = garch_model.fit(disp='off')
    garch_preds = np.sqrt(garch_res.forecast(horizon=horizon, reindex=False).variance.values.flatten()) / 100 * np.sqrt(252)
    all_garch_preds.append(pd.Series(garch_preds, index=y_test_fold.index, name="GARCH_Forecast"))

    gjr_model = arch_model(returns_loop, vol='Garch', p=1, o=1, q=1); gjr_res = gjr_model.fit(disp='off')
    gjr_preds = np.sqrt(gjr_res.forecast(horizon=horizon, reindex=False).variance.values.flatten()) / 100 * np.sqrt(252)
    all_gjr_preds.append(pd.Series(gjr_preds, index=y_test_fold.index, name="GJR_GARCH_Forecast"))

    # --- 4.3.3 **LSTM Models (Full and Simple)** ---
    print(f"Training LSTM models for Fold {i+1} (this may take a few minutes)...")

    # --- Full LSTM (with all features) ---
    X_train_loop_full = X_full.loc[current_train_df.index]
    y_train_loop = y.loc[current_train_df.index]
    X_test_loop_full = X_full.loc[current_test_chunk.index]

    fs_full = MinMaxScaler(); ts_full = MinMaxScaler()
    X_train_scaled_full = fs_full.fit_transform(X_train_loop_full)
    X_test_scaled_full = fs_full.transform(X_test_loop_full)
    y_train_scaled = ts_full.fit_transform(y_train_loop.values.reshape(-1, 1))

    X_train_seq, y_train_seq = create_sequences(X_train_scaled_full, y_train_scaled, WINDOW_SIZE)
    combined_X_for_seq = np.concatenate((X_train_scaled_full[-WINDOW_SIZE:], X_test_scaled_full))
    X_test_seq, _ = create_sequences(combined_X_for_seq, np.zeros(len(combined_X_for_seq)), WINDOW_SIZE)
    X_test_seq_full = X_test_seq[:len(X_test_loop_full)]

    model_full = build_lstm_model(WINDOW_SIZE, X_train_seq.shape[2])
    model_full.fit(X_train_seq, y_train_seq, epochs=EPOCHS_PROTOTYPE, batch_size=32, validation_split=0.15, callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)], verbose=0)
    lstm_preds_scaled = model_full.predict(X_test_seq_full)
    lstm_preds_full = ts_full.inverse_transform(lstm_preds_scaled)
    all_lstm_full_preds.append(pd.Series(lstm_preds_full.flatten(), index=y_test_fold.index, name="LSTM_Full_Forecast"))

    # --- Simple LSTM (with HAR features only) ---
    X_train_loop_simple = X_simple.loc[current_train_df.index]
    X_test_loop_simple = X_simple.loc[current_test_chunk.index]

    fs_simple = MinMaxScaler(); ts_simple = MinMaxScaler()
    X_train_scaled_simple = fs_simple.fit_transform(X_train_loop_simple)
    X_test_scaled_simple = fs_simple.transform(X_test_loop_simple)
    y_train_scaled_simple = ts_simple.fit_transform(y_train_loop.values.reshape(-1, 1))

    X_train_seq_s, y_train_seq_s = create_sequences(X_train_scaled_simple, y_train_scaled_simple, WINDOW_SIZE)
    combined_X_for_seq_s = np.concatenate((X_train_scaled_simple[-WINDOW_SIZE:], X_test_scaled_simple))
    X_test_seq_s, _ = create_sequences(combined_X_for_seq_s, np.zeros(len(combined_X_for_seq_s)), WINDOW_SIZE)
    X_test_seq_simple = X_test_seq_s[:len(X_test_loop_simple)]

    model_simple = build_lstm_model(WINDOW_SIZE, X_train_seq_s.shape[2])
    model_simple.fit(X_train_seq_s, y_train_seq_s, epochs=EPOCHS_PROTOTYPE, batch_size=32, validation_split=0.15, callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)], verbose=0)
    lstm_preds_scaled_s = model_simple.predict(X_test_seq_simple)
    lstm_preds_simple = ts_simple.inverse_transform(lstm_preds_scaled_s)
    all_lstm_simple_preds.append(pd.Series(lstm_preds_simple.flatten(), index=y_test_fold.index, name="LSTM_Simple_Forecast"))

print("\n--- Walk-Forward Validation Complete ---")

# ===================================================================
# PART 5: RESULTS AND EVALUATION
# ===================================================================
print("\n--- Comparing Model Performance ---")

# --- 5.1 Combine all results ---
har_results = pd.concat(all_har_preds); garch_results = pd.concat(all_garch_preds); gjr_results = pd.concat(all_gjr_preds)
lstm_full_results = pd.concat(all_lstm_full_preds); lstm_simple_results = pd.concat(all_lstm_simple_preds)
actuals_results = pd.concat(all_actuals); actuals_results.name = 'REALIZED_VOLATILITY'

results_df = pd.DataFrame(actuals_results)
results_df = results_df.join(har_results, how='left').join(garch_results, how='left').join(gjr_results, how='left')
results_df = results_df.join(lstm_full_results, how='left').join(lstm_simple_results, how='left')
results_df.dropna(inplace=True)

# --- 5.2 Calculate and Print Metrics ---
models_to_evaluate = ["HAR_Forecast", "GARCH_Forecast", "GJR_GARCH_Forecast", "LSTM_Full_Forecast", "LSTM_Simple_Forecast"]
metrics = {'Model': [], 'RMSE': [], 'MAE': []}
for model_name in models_to_evaluate:
    rmse = np.sqrt(mean_squared_error(results_df['REALIZED_VOLATILITY'], results_df[model_name]))
    mae = mean_absolute_error(results_df['REALIZED_VOLATILITY'], results_df[model_name])
    metrics['Model'].append(model_name)
    metrics['RMSE'].append(rmse)
    metrics['MAE'].append(mae)

metrics_df = pd.DataFrame(metrics)
print("\n--- Final Model Metrics (Walk-Forward) ---")
print(metrics_df.round(6))


# ===================================================================
# PART 6: ECONOMIC SIGNIFICANCE (HEDGING SIMULATION)
# ===================================================================
print("\n--- Part 6: Economic Significance (Hedging Simulation) ---")

volatility_threshold = y.loc[train_df.index].quantile(0.75)
print(f"Hedging Threshold (75th percentile of train vol): {volatility_threshold:.6f}")

results_df['Actual_Hedge'] = results_df['REALIZED_VOLATILITY'] > volatility_threshold
for model_name in models_to_evaluate:
    results_df[f'{model_name}_Hedge'] = results_df[model_name] > volatility_threshold

decision_metrics = {'Model': [], 'Hedge_Accuracy': []}
for model_name in models_to_evaluate:
    accuracy = accuracy_score(results_df['Actual_Hedge'], results_df[f'{model_name}_Hedge'])
    decision_metrics['Model'].append(model_name)
    decision_metrics['Hedge_Accuracy'].append(accuracy)

decision_metrics_df = pd.DataFrame(decision_metrics)
print("\n--- Hedging Decision Accuracy ---")
print(decision_metrics_df.round(4))

# ===================================================================
# PART 7: VISUALIZATION
# ===================================================================
plt.figure(figsize=(16, 8))
plt.plot(results_df['REALIZED_VOLATILITY'], label='Actual Volatility', color='black', linewidth=2)
plt.plot(results_df['HAR_Forecast'], label='HAR-RV (Linear)', color='green', alpha=0.7, linestyle=':')
plt.plot(results_df['LSTM_Simple_Forecast'], label='LSTM (Simple, HAR Features)', color='purple', alpha=0.8)
plt.plot(results_df['LSTM_Full_Forecast'], label='LSTM (Full, All Features)', color='red', alpha=0.8, linestyle='--')
plt.title('Walk-Forward Volatility Forecast Comparison')
plt.xlabel('Date')
plt.ylabel('Annualized Volatility')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()